# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [25]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [26]:
name = "mongoDB_testset"
file_name = f"clean_{name}.csv"
df = pd.read_csv(file_name)
df

,text_clean,label
0,hi this is my first jira ticket so please forg...,1
1,there must by a warning or an additional step ...,1
2,terrible example output you format the output ...,0
3,my fix to was incorrect but the normal test di...,0
4,starting in all database metadata indexes coll...,1
...,...,...
1851,seeing the document to run the unit tests i fi...,0
1852,the changes committed under autoyielding it sh...,0
1853,branch systemperfyml for change most reference...,1
1854,add support for parsing hosts specified as uni...,0


In [27]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label
20,NaN,1
29,NaN,0
34,NaN,0
117,NaN,0
137,NaN,0
211,NaN,0
290,NaN,0
331,NaN,0
382,NaN,0
423,NaN,1


In [28]:
df = df.dropna(subset=['text_clean'])
# reset index
df = df.reset_index(drop=True)
df

,text_clean,label
0,hi this is my first jira ticket so please forg...,1
1,there must by a warning or an additional step ...,1
2,terrible example output you format the output ...,0
3,my fix to was incorrect but the normal test di...,0
4,starting in all database metadata indexes coll...,1
...,...,...
1805,seeing the document to run the unit tests i fi...,0
1806,the changes committed under autoyielding it sh...,0
1807,branch systemperfyml for change most reference...,1
1808,add support for parsing hosts specified as uni...,0


In [29]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [30]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,909
0,901


In [31]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [32]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [33]:
print(f"Training set size: {len(train)/len(df):.2%}")
print(f"Validation set size: {len(validate)/len(df):.2%}")
print(f"Test set size: {len(test)/len(df):.2%}")

Training set size: 80.00%
Validation set size: 10.00%
Test set size: 10.00%


In [34]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)
# Remove __index_level_0__ column
tds = tds.remove_columns("__index_level_0__")
vds = vds.remove_columns("__index_level_0__")
test_ds = test_ds.remove_columns("__index_level_0__")

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 181
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 1448
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 181
    })
})

In [35]:
# Uncomment this if not logged in.
#notebook_login()

In [36]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]